# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
# Import initial libraries
import pandas as pd
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the time library and the datetime module from the datetime library 
import timeit
import time
from datetime import datetime
# Import the requests library
import requests
# Import the API key by using dotenv instead of config.py
%load_ext dotenv
%dotenv
import os
wkey = os.getenv('W10_KEY')

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)

In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [4]:
# # Create an empty list for holding the cities
# cities = []
# # Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
# for coordinate in coordinates:
#     # Use the citipy module to identify the nearest city to the current coordinate
#     city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
#     # If the city is unique, then add it to our cities list.
#     if city not in cities:
#         cities.append(city)
# Create a list for holding the cities by using list comprehension and unique()
cities = [citipy.nearest_city(coordinate[0], coordinate[1]).city_name for coordinate in coordinates]
cities = pd.unique(cities)
len(cities)

742

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [5]:
# Assemble the OpenWeatherMap's API call
units = ["standard", "metric", "imperial"]
url = "http://api.openweathermap.org/data/2.5/weather?units=" + units[2] + "&appid=" + wkey
city_url = url + "&q=" + cities[0]
city_weather = requests.get(city_url).json()
city_weather

{'coord': {'lon': 18.4232, 'lat': -33.9258},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 56.25,
  'feels_like': 55.38,
  'temp_min': 50.41,
  'temp_max': 62.38,
  'pressure': 1016,
  'humidity': 81},
 'visibility': 10000,
 'wind': {'speed': 7, 'deg': 126, 'gust': 15.01},
 'clouds': {'all': 0},
 'dt': 1665197976,
 'sys': {'type': 2,
  'id': 2073005,
  'country': 'ZA',
  'sunrise': 1665202437,
  'sunset': 1665248015},
 'timezone': 7200,
 'id': 3369157,
 'name': 'Cape Town',
 'cod': 200}

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [6]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            
        # Append the city information into the city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Current Description": city_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                         })
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | cape town
Processing Record 2 of Set 1 | college
Processing Record 3 of Set 1 | esik
Processing Record 4 of Set 1 | vostok
Processing Record 5 of Set 1 | nikolskoye
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 8 of Set 1 | hermanus
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | georgetown
Processing Record 11 of Set 1 | sao filipe
Processing Record 12 of Set 1 | calbuco
Processing Record 13 of Set 1 | ghatsila
Processing Record 14 of Set 1 | nome
Processing Record 15 of Set 1 | bethel
Processing Record 16 of Set 1 | vilhena
Processing Record 17 of Set 1 | punta arenas
Processing Record 18 of Set 1 | scarborough
Processing Record 19 of Set 1 | atuona
Processing Record 20 of Set 1 | clyde
Processing Record 21 of Set 1 | chuy
Processing Record 22 of Set 1 | busselton
Processing Record 23 o

Processing Record 41 of Set 4 | dentsville
Processing Record 42 of Set 4 | tumannyy
City not found. Skipping...
Processing Record 43 of Set 4 | launceston
Processing Record 44 of Set 4 | eyl
Processing Record 45 of Set 4 | ayan
Processing Record 46 of Set 4 | kahului
Processing Record 47 of Set 4 | ilulissat
Processing Record 48 of Set 4 | batemans bay
Processing Record 49 of Set 4 | pierre
Processing Record 50 of Set 4 | touros
Processing Record 1 of Set 5 | longyearbyen
Processing Record 2 of Set 5 | chandbali
Processing Record 3 of Set 5 | goryachegorsk
Processing Record 4 of Set 5 | kaitangata
Processing Record 5 of Set 5 | talnakh
Processing Record 6 of Set 5 | saint-augustin
Processing Record 7 of Set 5 | svetlaya
Processing Record 8 of Set 5 | hammond
Processing Record 9 of Set 5 | kashi
Processing Record 10 of Set 5 | kalmunai
Processing Record 11 of Set 5 | umm lajj
Processing Record 12 of Set 5 | salalah
Processing Record 13 of Set 5 | aasiaat
Processing Record 14 of Set 5 | 

Processing Record 30 of Set 8 | alibag
Processing Record 31 of Set 8 | yuzhno-kurilsk
Processing Record 32 of Set 8 | chicama
Processing Record 33 of Set 8 | alta floresta
Processing Record 34 of Set 8 | port-gentil
Processing Record 35 of Set 8 | vardo
Processing Record 36 of Set 8 | san quintin
Processing Record 37 of Set 8 | baykit
Processing Record 38 of Set 8 | puerto escondido
Processing Record 39 of Set 8 | rungata
City not found. Skipping...
Processing Record 40 of Set 8 | berlevag
Processing Record 41 of Set 8 | nyurba
Processing Record 42 of Set 8 | sovetskiy
Processing Record 43 of Set 8 | morro bay
Processing Record 44 of Set 8 | warrington
Processing Record 45 of Set 8 | port macquarie
Processing Record 46 of Set 8 | gunjur
Processing Record 47 of Set 8 | bella union
Processing Record 48 of Set 8 | portland
Processing Record 49 of Set 8 | the valley
Processing Record 50 of Set 8 | kandiari
Processing Record 1 of Set 9 | namibe
Processing Record 2 of Set 9 | flinders
Proces

Processing Record 19 of Set 12 | planaltina
Processing Record 20 of Set 12 | hobyo
Processing Record 21 of Set 12 | kuche
City not found. Skipping...
Processing Record 22 of Set 12 | gharghoda
Processing Record 23 of Set 12 | alotau
City not found. Skipping...
Processing Record 24 of Set 12 | guarapari
Processing Record 25 of Set 12 | caravelas
Processing Record 26 of Set 12 | oranjemund
Processing Record 27 of Set 12 | oktyabrskoye
Processing Record 28 of Set 12 | ayolas
Processing Record 29 of Set 12 | svirsk
Processing Record 30 of Set 12 | mayskiy
Processing Record 31 of Set 12 | mao
Processing Record 32 of Set 12 | tutoia
Processing Record 33 of Set 12 | kailua
Processing Record 34 of Set 12 | winslow
Processing Record 35 of Set 12 | porto velho
Processing Record 36 of Set 12 | chernyshevskiy
Processing Record 37 of Set 12 | yarmouth
Processing Record 38 of Set 12 | laguna
Processing Record 39 of Set 12 | artyk
City not found. Skipping...
Processing Record 40 of Set 12 | thung son

In [7]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

692

## Add the weather data to a new DataFrame.

In [8]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [9]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,Cape Town,-33.9258,18.4232,62.38,overcast clouds,78,100,7.00,ZA
1,College,64.8569,-147.8028,42.58,broken clouds,87,75,4.61,US
2,Esik,43.3552,77.4525,51.22,overcast clouds,44,100,0.22,KZ
3,Vostok,46.4856,135.8833,44.56,few clouds,39,13,5.75,RU
4,Nikolskoye,59.7035,30.7861,50.81,broken clouds,75,53,11.79,RU
5,Rikitea,-23.1203,-134.9692,71.33,few clouds,72,14,23.24,PF
6,Hermanus,-34.4187,19.2345,53.80,overcast clouds,80,100,7.43,ZA
7,Ushuaia,-54.8000,-68.3000,33.01,light snow,95,99,4.70,AR
8,Georgetown,5.4112,100.3354,82.24,scattered clouds,84,40,0.00,MY
9,Sao Filipe,14.8961,-24.4956,78.15,scattered clouds,84,48,5.82,CV


In [10]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Current Description', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country'],
      dtype='object')

In [11]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
new_columns = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]

# Recreate the DataFrame by using the new column order
city_data_df = city_data_df[new_columns]

# Display sample data
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Cape Town,ZA,-33.9258,18.4232,62.38,78,100,7.00,overcast clouds
1,College,US,64.8569,-147.8028,42.58,87,75,4.61,broken clouds
2,Esik,KZ,43.3552,77.4525,51.22,44,100,0.22,overcast clouds
3,Vostok,RU,46.4856,135.8833,44.56,39,13,5.75,few clouds
4,Nikolskoye,RU,59.7035,30.7861,50.81,75,53,11.79,broken clouds
5,Rikitea,PF,-23.1203,-134.9692,71.33,72,14,23.24,few clouds
6,Hermanus,ZA,-34.4187,19.2345,53.80,80,100,7.43,overcast clouds
7,Ushuaia,AR,-54.8000,-68.3000,33.01,95,99,4.70,light snow
8,Georgetown,MY,5.4112,100.3354,82.24,84,40,0.00,scattered clouds
9,Sao Filipe,CV,14.8961,-24.4956,78.15,84,48,5.82,scattered clouds


In [12]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [13]:
# Set the output file name
output_file = "../Weather_Database/WeatherPy_Database.csv"

# Export the city_data DataFrame into a CSV file
city_data_df.to_csv(output_file, index=False)